In [1]:
import os
import pandas as pd
from glob import glob
import json
import fnmatch
#import qgrid
import urllib.request, json
import datetime

## Check the ES-DOC errata pages for updates
- make 'csv/errata-files.csv' for using to make the updated catalog (MakeCloudCat.ipynb)
https://es-doc.github.io/esdoc-errata-client/installation.html
- many questions remain:
  - does status=resolved mean these files have been REPLACED? same version?
  - what do the various severity levels really mean? seems to be different for different models
  - could make a list of changed/new issues
  - could go through all 200+ issues and evaluate - feasible? make new boolean column 'include'
  - how to more efficiently deal with issues with >1,000 files, etc

# Sometimes the esgissue does NOT WORK, then will need to go back to old way:
Navigate to:  https://errata.es-doc.org/static/index.html

- Use chrome to download 'Web Page single file' for each set of 100 issues into ES-DOC?.mht

/bin/grep "<tr id=" ES-DOC1.mht > issues.txt
    
/bin/grep "<tr id=" ES-DOC2.mht >> issues.txt

/bin/grep "<tr id=" ES-DOC3.mht >> issues.txt

In [2]:
# maybe we should delete contents of issues and dsets directories?
# os/system('rm -rf issues dsets')
#command = '/usr/local/python/anaconda3/envs/pangeo-Oct2019/bin/esgissue retrieve -i $PWD/issues -d $PWD/dsets'
command = 'esgissue retrieve -i $PWD/issues -d $PWD/dsets'
#os.system(command)

ifiles = glob('issues/issue*.json')
issues = [ifile.split('issue_')[1].split('.json')[0] for ifile in ifiles]
len(issues)

286

In [ ]:
df = pd.DataFrame(columns=['uid', 'title', 'description', 'project', 'severity', 'status','urls'])
df_list = []
for item,issue in enumerate(issues):
    file_dsets = 'dsets/dset_'+issue+'.txt'
    if not os.path.isfile(file_dsets):
        file_dsets = 'dsets/dset_none.txt'

    file_issue = 'issues/issue_'+issue+'.json'
    
    with open(file_issue) as json_file:
        dict_issue = json.load(json_file)
   
    try:
        dlist = dict_issue['urls']
    except:
        dict_issue['urls'] = []
    
    df = df.append(dict_issue,ignore_index=True)
    print(item,file_dsets)
    
    df_dsets = pd.read_csv(file_dsets,delim_whitespace=True,header=None)
    df_dsets = df_dsets.rename(columns={0: "file_id"}).set_index([df_dsets.index])
    df_list += [list(df_dsets.file_id.values)]

df['file_ids'] = df_list
df = df.rename(columns={"uid": "issue_uid"})

In [ ]:
df.to_csv('csv/ES-DOC_issues.csv', index=False)
df.issue_uid.nunique()

In [ ]:
keywords = ['issue_uid','source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version', 'file_id','status','severity','issue_url']
df_all = []
for index, row in df.iterrows():
    if index > 143:
        continue
    file_id = row['file_ids']
    issue = row['issue_uid']
    status = row['status']
    severity = row['severity']
    issue_url = "https://errata.es-doc.org/static/view.html?uid="+issue
    dfs = pd.DataFrame(columns=keywords)
    print(index, ': issue uid:', issue, ', number of affected files:',len(file_id))
    for file in file_id:
        try:
            [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = file.split('.')
            [grid_label,version] = grid_version.split('#')
            klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,file,status,severity,issue_url]
            kdict = dict(zip(keywords, klist))
            dfs = dfs.append(kdict,ignore_index=True)
            df_all += [dfs]
        except:   # one issue has a list of datasets with two lines that need to be split apart
            #print('splitting lines for:',file_id)
            [s1,s2,s3]=file.split('CMIP6')
            files = ['CMIP6'+s2,'CMIP6'+s3]
            for f in files:
                [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = f.split('.')
                [grid_label,version] = grid_version.split('#')
                klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,f,status,severity,issue_url]
                kdict = dict(zip(keywords, klist))
                dfs = dfs.append(kdict,ignore_index=True)
                df_all += [dfs]
df_expand0 = pd.concat(df_all,sort=False).drop_duplicates(subset =["file_id"])

In [ ]:
keywords = ['issue_uid','source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version', 'file_id','status','severity','issue_url']
df_all = []
for index, row in df.iterrows():
    if index <= 143:
        continue
    file_id = row['file_ids']
    issue = row['issue_uid']
    status = row['status']
    severity = row['severity']
    issue_url = "https://errata.es-doc.org/static/view.html?uid="+issue
    dfs = pd.DataFrame(columns=keywords)
    print(index, ': issue uid:', issue, ', number of affected files:',len(file_id))
    for file in file_id:
        try:
            [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = file.split('.')
            [grid_label,version] = grid_version.split('#')
            klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,file,status,severity,issue_url]
            kdict = dict(zip(keywords, klist))
            dfs = dfs.append(kdict,ignore_index=True)
            df_all += [dfs]
        except:   # one issue has a list of datasets with two lines that need to be split apart
            #print('splitting lines for:',file_id)
            [s1,s2,s3]=file.split('CMIP6')
            files = ['CMIP6'+s2,'CMIP6'+s3]
            for f in files:
                [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = f.split('.')
                [grid_label,version] = grid_version.split('#')
                klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,f,status,severity,issue_url]
                kdict = dict(zip(keywords, klist))
                dfs = dfs.append(kdict,ignore_index=True)
                df_all += [dfs]
df_expand1 = pd.concat(df_all,sort=False).drop_duplicates(subset =["file_id"])

In [ ]:
df_expand = pd.concat([df_expand0,df_expand1],sort=False).drop_duplicates(subset =["file_id"])
df_expand[['source_id','file_id']].groupby(['source_id']).count()
#df_expand

In [ ]:
import datetime
date = str(datetime.datetime.now().strftime("%Y%m%d"))
os.system('cp csv/errata-files.csv csv/errata-files-'+date+'.csv')
df_expand.to_csv('csv/errata-files.csv', mode='w+', index=False)